In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py

import numpy as np
import scipy

# Triangular solver

Based on scipy's [scipy.linalg.solve_triangular](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_triangular.html) and the solver code presented by [Weng et al. at HPCA'20](https://ieeexplore.ieee.org/abstract/document/9065593)

Consider the following system of equations:

$$
\begin{align}
a_{11} x_1&              &= b_1 \\
a_{21} x_1&+ a_{22} x_2 &= b_2
\end{align}
$$

This can be expressed as a matrix equation $\mathbf{A}x=b$ where
$
\mathbf{A} = \left[\begin{array}{c c}
  a_{11} & 0      \\
  a_{21} & a_{22}
  \end{array}
  \right]$
and $
b = \left[\begin{array}{c}
  b_1 \\ b_2 \end{array} \right]
$.

To solve, you would conclude that $x_1 = b_1 / a_{11}$, and plug it into the second equation to solve for $x_2$. However, this would correspond to a column-major traversal of $\mathbf{A}$, so we transpose $\mathbf{A}$ to get a row-major (and hopefully concordant) traversal.

(Evidence of this transposition manifests below as generating `A_data` as an _upper_ triangular matrix, as well as the `trans="T"` keyword argument passed to `solve_triangular()` in the assertion.)

_Note:_ this was developed expecting uncompressed tensors but its use of `getPayload{,Ref}()` should support compressed formats.

In [ ]:
M = N = 4

np.random.seed(12345)

# Generate a random upper triangular matrix A
A_data = np.triu(np.random.randint(1, 6, size=(M, N)))

# Generate the desired output vector x (Ax=b)
X_ref_data = np.array(np.random.randint(1, 6, size=(N,)))

# Evaluate the triangular matrix with X_ref to obtain B
#   this corresponds to a column-wise dot product with 
B_data = X_ref_data.dot(A_data)

print("A:\n", A_data, "\nx (reference):\n", X_ref_data, "\nb:\n", B_data)

assert(all(X_ref_data == scipy.linalg.solve_triangular(A_data, B_data, trans="T")))

In [ ]:
# Create uncompressed tensors
A_MN = Tensor.fromUncompressed(["M", "N"], A_data.tolist()).setName("A")
B_N = Tensor.fromUncompressed(["N"], B_data.tolist()).setName("b")
X_ref = Tensor.fromUncompressed(["M"], X_ref_data.tolist()).setName("x_ref")
X = Tensor(rank_ids=["M"]).setName("x")

A = A_MN.getRoot()

inflate = False
if inflate:
    # hack to "inflate" A with coordinates where there are zeros,
    # and allowing us to use square brackets (i.e. __getitem__)
    # because positions == coordinates
    A << Fiber(coords=range(M))
    for j, a_j in A:
        a_j << Fiber(coords=range(N))

B = B_N.getRoot()

displayTensor(A)
displayTensor(B)

## Pipelined triangular solver

`P` is a representation of pipelined communication; each row corresponds to the data passed from the previous stage to the next. Each row is N elements wide (same as B).

This mapping corresponds to a pipeline of single-threaded compute engines (CEs) commensurate in size to the input.

_Note:_ I traverse A in the N rank, then the M rank. I think I need a `swapRanks()` somewhere...

In [ ]:
# create "pipes" between each stage: N-entry tensors
# (can't simply do a rank-2 tensor because I'd have multiple entries for addActivity for that tensor, and
# current pipelining API doesn't really handle that
#
# for syntactic cleanliness I include B, the input tensor, in this array too
BP = [B_N] + [Tensor(rank_ids=["N"], shape=[N]).setName(f"pipe-{m}") for m in range(M)]

canvas = createCanvas(A_MN, *BP, X, enable_wait=True)

cycle = 0
stage_delay = 1

# "wait" means, for each key-value pair in the supplied dictionary,
# we must wait `value` cycles when the `key`th argument in the canvas has been updated 
# in order to add our own activity
wait = None # loop-carried variable

for a_m in range(M):
    # in the first row, read from B, rather than P
    # also, skew these starts by cycle, as these input elements are piped in one after another
    # m_ and n_activities provide pair-wise activity locations for adjacent stages in the pipeline
    # e.g., [(m,), (m,), ()] then [(), (m,), (m,)] then [(), (), (m,)]
    m_activities = [(a_m,) if mm == a_m or mm == a_m+1 else () for mm in range(M+1)]
    # print(m_activities)

    # process element on diagonal
    curStage = BP[a_m]
    nextStage = BP[a_m+1]
    a_n = a_m
    next_m = nextStage.getPayloadRef(a_m)
    next_m <<= curStage.getPayload(a_m) / A.getPayload(a_m, a_n)
    addActivity(canvas, (a_m, a_n), *m_activities, worker=str(a_m), skew=(cycle if a_m == 0 else 0), wait=wait)
    cycle += 1
    
    # process elements off-diagonal on same row
    # change from a_m to a_n (even though M=N) for clarity
    for i_n in range(a_n+1, N):
        n_activities = [(i_n,) if ii == a_n or ii == a_n+1 else () for ii in range(N+1)]
        # print(n_activities)
        
        cur_n = curStage.getPayload(i_n)
        next_n = nextStage.getPayloadRef(i_n)
        next_n <<= cur_n - next_m * A.getPayload(a_m, i_n)

        addActivity(canvas, (a_m, i_n), *n_activities, worker=str(a_m), skew=(cycle if a_m == 0 else 0), wait=wait)
        cycle += 1
        
    # new values for next value of a_m
    wait = {f"pipe-{a_m}": stage_delay}

for x_m in range(M):
    x_m_ref = X.getPayloadRef(x_m)
    x_m_ref <<= BP[x_m+1].getPayload(x_m) # get m-th value from pipeline stage m
    m_activities = [(x_m,) if mm == x_m else () for mm in range(M)]
    # print(x_m, [], [], m_activities + [(x_m,)])
    addActivity(canvas, [], [], *m_activities, (x_m,), wait={f"pipe-{x_m}":stage_delay})
            
displayCanvas(canvas)


In [ ]:
# final result
displayTensor(X)
displayTensor(X_ref)
X == X_ref

## Check results

Check the result by performing the dot product of `A` and `x` and ensuring that equals the `b` we started with. (The jth entry of `b` corresponds to the dot product of the jth _column_ of `A` and `x`.)

In [ ]:
B_check = Fiber(coords=range(N), initial=0)
# displayTensor(B2)

A_NM = A_MN.swapRanks()
A2 = A_NM.getRoot()
displayTensor(A2)

# canvas2 = createCanvas(A, X, B2)
for a2_n in range(N):
    for a2_m in range(M):
        # print(j, i, a_pay[i], x_val, b2_ref)
        b2_ref = B_check.getPayloadRef(a2_n)
        x_val = X.getPayload(a2_m)
        b2_ref += A2.getPayload(a2_n, a2_m) * x_val 
        # addActivity(canvas2, (i,j), (i,), (i,))
        
# displayCanvas(canvas2)

# check results
B == B_check

# Tiling schemes

For your convenience, `A_MN` and `B_N` are repeated here:

In [ ]:
displayTensor(A_MN)
displayTensor(B_N)

numPEs = 2
numLanes = 2

### A_MMNN (due to Joel)

The program above must have `M` PEs for an MxM matrix. This is not tenable for large matrices, and also, such a pipeline would have load imbalance across the PEs: the first PE sees all elements of B, but the last sees just one.

Tiling helps most immediately with the first problem, by splitting the problem into tiles, each of which can be solved by a system with a bounded number of PEs.

Tiling is tantamount to creating an additional rank. Suppose we want to tile `A_MN` so that an `M0`-sized subset of rows can be processed by `M0` PEs at a time. We'll call `splitUniform(M0)` to produce a rank-3 tensor with ranks `M.1`, `M.0`, and `N`, called `A_MMN`. No fiber in rank `M.0` will contain more than `M0` payloads.

If your ranks are all spatial (in other words, not temporal, as pipeline parallelism will do), then splitting one rank won't be enough. This is because `N` could possibly be unbounded, and each PE only has a finite number of elements (i.e., lanes) it can process at a time. This is why we also split the `N` rank with `splitUniform(N0)` so that each PE can process `N0` elements at a time in its `N0` lanes.

This is how we produce a rank-4 tensor A_MMNN:

In [ ]:
M0 = numPEs
N0 = numLanes

A_MMNN = A_MN.splitUniform(M0).splitUniform(N0, depth=2)
displayTensor(A_MMNN)

### A_MNMN

The convention is to order ranks such that any ranks that are "scale-up" or unbounded appear closer to the root, and any bounded ranks further from them. This is called _normal form_.

The jury is still out as to whether time-based ranks should appear above or below space-based ranks.

To bring it to normal form, the `swapRanks()` at `depth=1` moves the bounded ranks to the bottom (conversely, unbounded ranks to the top).

In [ ]:
A_MNMN = A_MMNN.swapRanks(depth=1)
displayTensor(A_MNMN)

### A_MMMN (due to Neal)
M0 in the number of lanes, M1 in the number of PEs, and all N elements streamed in time

In [ ]:
A_MMMN = A_MN.splitUniform(numPEs).splitUniform(numLanes)
# M.1.1 is "M2", which  and M.1.0 is "M1"
A_MMMN.ranks[0].id = "M.2"
A_MMMN.ranks[1].id = "M.1"
displayTensor(A_MMMN)

### A_MNMM
To bring the unbounded rank (N) above bounded ranks (here, M1 for PEs, and M0 for lanes), need to swap N and M0 (depth 2), then N and M1 (depth 1).

Note that this would use B_N, not B_NN (below).

In [ ]:
A_MNMM = A_MMMN.swapRanks(depth=2).swapRanks(depth=1)
displayTensor(A_MNMM)

### B_NN

In [ ]:
B_NN = B_N.splitUniform(N0)
displayTensor(B_NN)

### X_MM (output for Joel's version)

In [ ]:
X_MM = Tensor(rank_ids=["M1", "M0"])
displayTensor(X_MM)

### Fibertree Native Traversal

In [ ]:
P = Tensor(rank_ids=["M", "N"], name = "P")
X = Tensor(rank_ids=["M"], name = "X")

a_m = A_MN.getRoot()
b_n = B_N.getRoot()
p_m = P.getRoot()
x_m = X.getRoot()

# Initialize M[0] of P to be B, then don't use B anymore.
p_n0 = P.getRoot().getPayloadRef(0)
for n, (p_ref, b_val) in p_n0 << b_n:
    p_ref <<= b_val

canvas = createCanvas(A_MN, P, X)
    
for m, (x_ref, (p_n, a_n)) in x_m << (p_m << a_m):
    p_n_prime = p_m.getPayloadRef(m+1)
    is_first = True
    prop_val = 0
    for n, (p_ref, (p_val, a_val)) in p_n_prime << (p_n & a_n):
        if is_first:
            prop_val = p_val / a_val
            x_ref <<= prop_val 
            is_first = False
            addFrame(canvas, (m, n), (m, n), (m,))
        else:
            p_ref <<= p_val - prop_val * a_val
            addFrame(canvas, (m, n), [(m, n), (m+1, n)], ())

#displayTensor(A_MN)
#displayTensor(P)
#displayTensor(X)
displayCanvas(canvas)

### MNMN traversal (thanks to Michael Pellauer) (WIP)

In [ ]:
P_MNMN = Tensor(rank_ids=["M1", "N1", "M0", "N0"], name = "P")
X_MM = Tensor(rank_ids=["M1", "M0"], name = "X")

a_m1 = A_MNMN.getRoot()
b_n1 = B_NN.getRoot()
p_m1 = P_MNMN.getRoot()
x_m1 = X_MM.getRoot()

# Initialize M1[0] of P to be B, then don't use B anymore.
p_n1_0 = P_MNMN.getRoot().getPayloadRef(0)
for n1, (p_m0, b_n0) in p_n1_0 << b_n1:
    p_n0_0 = p_m0.getPayloadRef(0)
    for n0, (p_ref, b_val) in p_n0_0 << b_n0:
        p_ref <<= b_val

#displayTensor(A_MNMN)
#displayTensor(P_MNMN)

canvas = createCanvas(A_MNMN, P_MNMN, X_MM)

# note leading variable in for loop matches order of traversal
# use << when it's an output
for m1, (x_m0, (p_n1, a_n1)) in x_m1 << (p_m1 << a_m1):
    is_first = [True for m0 in range(M0)]
    prop_val = [0 for m0 in range(M0)]
    for n1, (p_m0, a_m0) in p_n1 & a_n1:
        for m0, (x_ref, (p_n0, a_n0)) in x_m0 << (p_m0 << a_m0):
            m = m0
            next_m = m+1
            next_m1 = (next_m // M0) * M0
            next_m0 = next_m
            rel_m0 = m % M0
            #print(f"{m1}, {m0}")
            p_n0_prime = p_m1.getPayloadRef(next_m1, n1, next_m0)
            for n0, (p_ref, (p_val, a_val)) in p_n0_prime << (p_n0 & a_n0):
                if is_first[rel_m0]:
                    prop_val[rel_m0] = p_val / a_val
                    x_ref <<= prop_val[rel_m0]
                    is_first[rel_m0] = False
                    addFrame(canvas, (m1, n1, m0, n0), (m1, n1, m0, n0), (m1, m0))
                else:
                    p_ref <<= p_val - prop_val[rel_m0] * a_val
                    addFrame(canvas, (m1, n1, m0, n0), [(m1, n1, m0, n0), (next_m1, n1, next_m0, n0)], ())
#displayTensor(P_MNMN)
displayCanvas(canvas)

### X_MMM (output for Neal's version)

In [ ]:
X_MMM = Tensor(rank_ids=["M2", "M1", "M0"])
displayTensor(X_MMM)